In [1]:
import requests
import json
import pandas
import datetime
import matplotlib.pyplot as plt

In [ ]:
Exchange_Info_API = 'https://api.binance.com/api/v3/exchangeInfo'
Price_Info_API = 'https://api.binance.com/api/v3/klines'

In [ ]:
def symbols_info():
    data = json.loads(requests.get(Exchange_Info_API).text)
    cols = ['symbol','status','baseAsset','quoteAsset']
    data = [[i[j] for j in cols] for  i in data['symbols']]
    return pandas.DataFrame(data,columns=cols) 

In [ ]:
def Date_to_timestamp(_time,_format = '%d/%m/%Y %H:%M:%S'):
    return int(datetime.datetime.timestamp(datetime.datetime.strptime(_time,_format ))*1000)

In [ ]:
def Price_info(symbol,step='1m',start=None,end=None,limit=500, base = '',quote='') :

    if start != None and type(start) == str:
        start = Date_to_timestamp(start)
    if end != None and type(end) == str:
        end = Date_to_timestamp(end)
    

    par = {
        'symbol':symbol,
        'startTime':start,
        'endTime':end,
        'interval':step,
        'limit':limit}


    cols =[
        'Kline open time',   #if a range of time
        'Open price',       #First trade price in the time range
        'High price',       #max buy in time
        'Low price',        #Min sell in time
        'Close price',      #Last trade price in the time range
        'Volume',           #Sum in time
        'Kline Close time',
        'Quote asset volume',   #Sum in time
        'Number of trades',     #Sum in time
        'Taker buy base asset volume',  #Sum in time
        'Taker buy quote asset volume', #Sum in time
        'Unused field, ignore'] # not use 
    p=pandas.DataFrame(json.loads(requests.get(Price_Info_API,params=par).text),columns=cols)
    p.insert(0,'quote', quote)
    p.insert(0,'base', base)
    p.insert(0,'Symbol', symbol)

    # base/quote = price
    return p

In [ ]:
symbols = symbols_info()
data = pandas.concat(
   [Price_info(
       i['symbol'],
       '30m',
       '26/11/2022 00:00:00',
       '14/12/2022 00:00:00',
       1000,
       i['baseAsset'],
       i['quoteAsset']
       ) for _,i in symbols[symbols['status']=='TRADING'].iterrows()
   ],
   ignore_index=True)

symbols.to_csv("Data\symbols.csv")
data.to_csv("Data\data.csv")